## Fluid flow model of the queue for shelter and housing

Here we ignore the randomness in the arrival process and the service process for homeless people entering and leaving the homeless response system. Instead we assume that ``fluid" flows into the system continuously at a rate $\lambda(t)$ and flows out a  rate $\mu(t) = \mu_{0}n_{h}(t)$ where $\mu_{0}$ is the service rate of a single housing unit and $n_{h}(t)$ is the number of houses at time $t$. The simplification of the service process relies on the assumption that the servers are always busy. Given a house building rate of $h(t)$, we can calculate $n_{h}(t)$ as: 

$n_h(t) = n_{h0} + \int_{0}^{t} h(t) dt$

where $n_{h0}$ is the initial number of houses in the system. At time $t$ we can calculate the number of people $X(t)$ in the system as:

$X(t) = X_0 + \int_{0}^{t} \lambda(t) dt - \int_{0}^{t} \mu(t) dt$.

We split the queue for housing into an unsheltered and a sheltered part. We denote $n_{s}(t)$ as the number of shelters at time $t$. Given a server building rate of $s(t)$ and and initial number of shelters in the system $n_{s0}$ we can calculate $n_{s}(t)$ as:

$n_s(t) = n_{s0} + \int_{0}^{t} s(t) dt$. 

The size of the unsheltered queue at time $t$ is then: 

$unsh(t) = X(t) - n_h(t) - n_s(t) = X_0 + \int_{0}^{t} \lambda(t) dt - \int_{0}^{t} \mu(t) dt - n_h(t) - n_s(t)$

where we assume that build rates $h(t)$ and $s(t)$ are sufficiently small for the given arrival rate $\lambda(t)$ so that the expression above for $unsh(t)$ never drops below $0$. The purpose of this assumption is to ensure that an objective function which includes $unsh(t)$ does not involve an expression containing a maximisation (e.g. $\max(0,X(t)-n_h(t)-n_s(t))$). 

We can use this deterministic model to estimate statistical variability using: 

$E[unsh(t)] = X_0 + \int_{0}^{t} \lambda(t) dt - \int_{0}^{t} \mu(t) dt - n_h(t) - n_s(t)$

$Var[unsh(t)] = \int_{0}^{t} \lambda(t) dt + \int_{0}^{t} \mu(t) dt$

where the latter expression assumes exponential inter-arrival and service times. The above expressions can then help us calculate squared queue lengths: 

$E[unsh(t)^2] = E[unsh(t)]^2 + Var[unsh(t)]$. 

## Deterministic optimisation using fluid flow model

The fluid flow model has the advantages that:

* it is simple
* it is quick to run 
* it accepts building rates from the set of non-negative real numbers (useful in a continuous optimisation framework). 

We now use this fluid flow model to compute deterministic objective values in an continuous optimisation formulation. We will solve this continuous problem using Pyomo which is able to solve problems with 'black-box' objective functions. Note: we solved a discrete version of this problem previously using the analytical $M(t)/M/s(t)$ queueing model and a complete enumeration of the feasible region. 

#### Model setup

In [2]:
import fluid_flow_model as fl
import json
import matplotlib.pyplot as plt

In [3]:
# Opening JSON file
with open('data_as_is_exp1.json') as json_file:
    data_as_is_exp1 = json.load(json_file)

In [4]:
print('Length of model run (yrs): ' + str(data_as_is_exp1['analysis_horizon']))
print('Initial capacity: ' + str(data_as_is_exp1['initial_capacity']))
print('Initial demand: ' + str(data_as_is_exp1['initial_demand']))
for i in range(len(data_as_is_exp1['arrival_rates'])):
    print('Arrival rates yr ' + str(i+1) + ': ' + str(data_as_is_exp1['arrival_rates'][i]))

Length of model run (yrs): 4
Initial capacity: {'housing': 40, 'shelter': 15}
Initial demand: 180
Arrival rates yr 1: 100
Arrival rates yr 2: 100
Arrival rates yr 3: 100
Arrival rates yr 4: 100


Let $\mu_0$ be the housing service rate for a single housing unit. 

Let $T \in Z^{+}$ be a fixed horizon over which to model queue behaviour and to make building decisions.

Let vectors $\boldsymbol{h} = \{h_t \hspace{0.2cm} \forall t \in {1,...,T}\}$ and $\boldsymbol{s} = \{s_t \hspace{0.2cm} \forall t \in {1,...,T}\}$ denote annual house and shelter building rates, respectively. For simplicity we say that housing/shelter building rates are constant within each year. 

Let $c_{h} = 1$ be the cost of increasing $h_t$ by one, for any $t$.

Let $c_{s}$ be the cost of increasing $s_t$ by one, for any $t$. 

Let $y(\boldsymbol{h},\boldsymbol{s})$ be a deterministic objective function, evaluated using the fluid flow model. 

Let $B$ be a baseline minimum annual house/shetler building rate

Let $C$ be a total budget for building housing and shelter

#### Optimisation formulation

$\min_{\boldsymbol{h},\boldsymbol{s}} \hspace{0.2cm} y(\boldsymbol{h},\boldsymbol{s})$

$\text{ s.t.} \hspace{0.8cm} \sum_{t=1}^{T} c_{h}h_{t} + c_{s}s_{t} \leq B$

$\hspace{1.6cm} h_t, s_t \geq B \hspace{0.2cm} \forall t \in \{1, ..., T\}$

As an example, we set the objective function to be the time average of the expected value of the unsheltered queue: 

$y(\boldsymbol{h},\boldsymbol{s}) = \frac{1}{T} \int_0^T E[unsh(t; \boldsymbol{h},\boldsymbol{s})] dt$

We set: 

$\mu_0 = 0.25$

$T=4$

$c_s = 0.5$

$B=12$

$C = B (c_h + c_s) T + 60$. 

This value of $C$ allows an extra budget of $60$ to be spent at any time on either shelter or housing. This could, for example, afford an extra build rate of $60$ houses/yr or $120$ shelters/yr in a specific year, or some mixture of extra housing and extra shelter.

In [2]:
# Opening JSON file
with open('data_as_is.json') as json_file:
    data_as_is = json.load(json_file)

In [4]:
model = fl.FluidFlowModel(data_as_is, data_as_is['build_rates'])

In [5]:
horizon = 5
T = [i/365 for i in range(int(horizon*365))]
model.analyse(T)

In [ ]:
plt.plot(T, model.unsh_t, color = 'blue')
plt.title('E[Unsheltered] - fluid flow model')
plt.xlabel('t (years)')
plt.ylabel('Obj value: E[Unsheltered]')
plt.show()

In [ ]:
plt.plot(T, model.unsh_var_t, color = 'blue')
plt.title('Var[Unsheltered] - fluid flow model')
plt.xlabel('t (years)')
plt.ylabel('Var[Unsheltered]')
plt.show()

In [ ]:
plt.plot(T, model.unsh_sq_t, color = 'blue')
plt.title('E[Unsheltered^2] - fluid flow model')
plt.xlabel('t (years)')
plt.ylabel('E[Unsheltered^2]')
plt.show()